In [1]:
from pyVHR.datasets.ubfc2 import UBFC2
from pyVHR.datasets.dataset import Dataset
from pyVHR.datasets.dataset import datasetFactory
from pyVHR.methods.base import methodFactory
from pyVHR.signals.video import Video
from copy import copy
import numpy as np

import os
#RUN ON CPU 
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

#Tensorflow/KERAS
import tensorflow as tf
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.callbacks import ModelCheckpoint
from tensorflow.python.keras.models import model_from_json
from tensorflow.python.keras.layers import ZeroPadding3D, Dense, Activation,Conv3D,MaxPooling3D,AveragePooling3D,Flatten,Dropout
from tensorflow.python.keras.utils import np_utils
from tensorflow.python.keras.models import model_from_json


In [6]:
# -- dataset object
dataset = UBFC2()

# -- ground-truth (GT) signal
fname = "./UBFC/DATASET_2/subject1/ground_truth.txt"

# -- load signal and build a BVPsignal or ECGsignal object
sigGT = dataset.readSigfile(fname)

# -- compute BPM GT
winSizeGT = 2
bpmGT, timesGT = sigGT.getBPM(winSizeGT)


In [7]:
bpm = np.round(bpmGT)
bpm =bpm - 55
bpm = np.round(bpm / 2.5)
print(bpm)

[-2. 12. 16. 15. 18. 16. 18. 14. 18. 18. 22. 20. 26. 16. 18. 20. 20. 19.
 22. 23. 24. 20. 18. 22. 26. 19. 18. 20. 15. 19. 21. 23. 24. 21. 22. 18.
 22. 17. 23. 17. 24. 23. 25. 20. 15. 20. 18. 21. 23. 22. 20. 22. 19. 21.
 19.]


In [8]:
print(timesGT)

[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.
 18. 19. 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 30. 31. 32. 33. 34. 35.
 36. 37. 38. 39. 40. 41. 42. 43. 44. 45. 46. 47. 48. 49. 50. 51. 52. 53.
 54.]


In [8]:
LENGTH_VIDEO = 60 
IMAGE_WIDTH = 25 
IMAGE_HEIGHT = 25 
IMAGE_CHANNELS = 1 
RATE = 30
NB_SECOND = int(LENGTH_VIDEO / RATE)
# Available Outputs
HEART_RATES = np.linspace(55, 240, 75)
NB_CLASSES = len(HEART_RATES)


def extractDataFromVideo(videoFilename, GTFilename):
    
    sigGT = dataset.readSigfile(GTFilename)
    winSizeGT = NB_SECOND
    bpmGT, timesGT = sigGT.getBPM(winSizeGT)
    
    bpm = np.round(bpmGT)
    bpm = bpm - 55
    bpm = np.round(bpm / 2.5)
    
    video = Video(videoFilename)
    video.getCroppedFaces(detector='dlib', extractor='skvideo')
    video.setMask(typeROI='skin_adapt',skinThresh_adapt=0.22)

    NB_LAPSE = int(video.numFrames / RATE)

    imgs = np.zeros(shape=(video.numFrames, video.cropSize[0], video.cropSize[1], 1))
    xtest = np.zeros(shape=(NB_LAPSE, LENGTH_VIDEO, IMAGE_HEIGHT , IMAGE_WIDTH, 1))
    ytest = np.zeros(shape=(NB_LAPSE, NB_CLASSES + 1))

    # prepare labels and label categories
    labels = np.zeros(NB_CLASSES + 1)

    for i in range(NB_CLASSES + 1):
        labels[i] = i
    labels_cat = np_utils.to_categorical(labels)
 
    # channel extraction
    if (video.cropSize[2]<3):
        IMAGE_CHANNELS = 1
    else:
        IMAGE_CHANNELS = video.cropSize[2]

    # load images (imgs contains the whole video)
    for j in range(video.numFrames):

        if (IMAGE_CHANNELS==3):
            temp = video.faces[j]/255
            temp = temp[:,:,1]      # only the G component is currently used
        else:
            temp = video.faces[j] / 255

        imgs[j] = np.expand_dims(temp, 2)
    


    for lapse in range(0,NB_LAPSE):  
        xtemp = np.zeros(shape=(LENGTH_VIDEO, IMAGE_HEIGHT , IMAGE_WIDTH, 1)) 
    
        start = lapse * RATE
        end = start + LENGTH_VIDEO
        if(end > video.numFrames):
            break
        c=0
    
        for j in range(start,end-1):    
            faceCopy = copy(imgs[j])
            #randomization pixel choice 
            np.random.shuffle(faceCopy)
            for m in range(0, IMAGE_HEIGHT):
                for n in range(0, IMAGE_WIDTH):
                    xtemp[c][m][n]= faceCopy[m][n]
            c = c +1
                
        xtest[lapse] = np.expand_dims(xtemp, 0)
        ytest[lapse] = np.expand_dims(labels_cat[int(bpm[lapse+NB_SECOND])],0)
    return xtest, ytest

In [20]:
dataset = datasetFactory("UBFC2")

xtrain = np.array(np.zeros(shape=(0,LENGTH_VIDEO, IMAGE_HEIGHT, IMAGE_WIDTH, 1)))
ytrain = np.zeros(shape=(0, NB_CLASSES + 1))

for i in range (len(dataset.videoFilenames)):
    xtest, ytest = extractDataFromVideo(dataset.videoFilenames[i], dataset.sigFilenames[i])
    xtrain = np.concatenate((xtrain, xtest), axis=0)
    ytrain = np.concatenate((ytrain, ytest), axis=0)

indices = np.arange(xtrain.shape[0])
np.random.shuffle(indices)

xtrain = xtrain[indices]
ytrain = ytrain[indices]

np.savez('./trainingData.npz', a=xtrain, b=ytrain)
print(np.shape(xtrain))

(1629, 60, 25, 25, 1)


In [21]:
model = model_from_json(open('./model/model_conv3D.json').read())
model.load_weights('./model/weights_conv3D.h5')
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


#start training
model.fit(xtrain, ytrain, epochs = 10, batch_size=256, verbose=1)


Epoch 1/10
7/7 [==============================] - 32s 5s/step - loss: 1390.7048 - accuracy: 0.0153
Epoch 2/10
7/7 [==============================] - 32s 5s/step - loss: 613.5331 - accuracy: 0.0362
Epoch 3/10
7/7 [==============================] - 32s 5s/step - loss: 336.6241 - accuracy: 0.0270
Epoch 4/10
7/7 [==============================] - 32s 5s/step - loss: 204.1564 - accuracy: 0.0313
Epoch 5/10
7/7 [==============================] - 32s 5s/step - loss: 135.8026 - accuracy: 0.0368
Epoch 6/10
7/7 [==============================] - 32s 5s/step - loss: 101.0680 - accuracy: 0.0417
Epoch 7/10
7/7 [==============================] - 34s 5s/step - loss: 75.9226 - accuracy: 0.0405
Epoch 8/10
7/7 [==============================] - 32s 5s/step - loss: 58.0394 - accuracy: 0.0399
Epoch 9/10
7/7 [==============================] - 33s 5s/step - loss: 41.6665 - accuracy: 0.0307
Epoch 10/10
7/7 [==============================] - 38s 5s/step - loss: 30.8884 - accuracy: 0.0270


In [22]:
# DEFINE MODEL
model = Sequential()

#feature extraction
model.add(Conv3D(filters=32, kernel_size=(LENGTH_VIDEO-2,IMAGE_HEIGHT-5,IMAGE_WIDTH-5), input_shape=(LENGTH_VIDEO, IMAGE_HEIGHT, IMAGE_WIDTH, 1)))
model.add(MaxPooling3D(pool_size=(2,2,2)))
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Flatten())

#Classification
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(NB_CLASSES + 1, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

#start training
model.fit(xtrain, ytrain, epochs = 10, batch_size=256, verbose=1)

Epoch 1/10
7/7 [==============================] - 35s 5s/step - loss: 5.7733 - accuracy: 0.0405
Epoch 2/10
7/7 [==============================] - 43s 6s/step - loss: 4.2402 - accuracy: 0.0540
Epoch 3/10
7/7 [==============================] - 37s 5s/step - loss: 4.2166 - accuracy: 0.0565
Epoch 4/10
7/7 [==============================] - 41s 6s/step - loss: 4.1854 - accuracy: 0.0546
Epoch 5/10
7/7 [==============================] - 33s 5s/step - loss: 4.1447 - accuracy: 0.0675
Epoch 6/10
7/7 [==============================] - 33s 5s/step - loss: 4.0925 - accuracy: 0.0620
Epoch 7/10
7/7 [==============================] - 33s 5s/step - loss: 4.0292 - accuracy: 0.0546
Epoch 8/10
7/7 [==============================] - 33s 5s/step - loss: 3.9565 - accuracy: 0.0645
Epoch 9/10
7/7 [==============================] - 33s 5s/step - loss: 3.8758 - accuracy: 0.0528
Epoch 10/10
7/7 [==============================] - 33s 5s/step - loss: 3.7953 - accuracy: 0.0675
